# Alice's LLM Remote Example

This notebook demonstrates how Alice can set up her data, grant permissions to Bob, and use the LLM remote functionality.

In [1]:
from __future__ import annotations

import os
import time
from pathlib import Path
from loguru import logger

# Import the client module
from llama_remote import client

# Helper function to create sample files
def create_sample_file(file_path, content):
    """Create a sample file with the given content."""
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as f:
        f.write(content)
    print(f"Created sample file: {file_path}")

In [2]:
# Creating Alice's client
alice_client = client("~/.syft_alice_config.json")
print(f"Connected as: {alice_client.client.email}")

Connected as: alice@openmined.org


## Create Alice's Private Medical Data

In [3]:
# Alice's private data - medical research
alice_data_dir = str(alice_client.box.client.datasite_path) + "/private_data"
alice_files = {
    f"{alice_data_dir}/patient_data.csv": """
patient_id,age,gender,condition,treatment,outcome
P001,45,F,Hypertension,Medication A,Improved
P002,67,M,Diabetes,Medication B,Stable
P003,33,F,Anxiety,Therapy C,Significantly Improved
P004,52,M,Arthritis,Medication D,No Change
P005,29,F,Depression,Therapy E,Improved
""",
    f"{alice_data_dir}/research_notes.txt": """
CONFIDENTIAL RESEARCH NOTES
Date: 2025-03-01

Our preliminary findings suggest that the new treatment protocol shows a 35% improvement 
in patient outcomes compared to the standard approach. Side effects were minimal 
and reported in only 7% of participants.

Key observations:
- Treatment efficacy increases with consistent administration
- Age group 40-55 showed the most significant improvement
- No correlation found between gender and treatment response

Next steps: Expand trial to 200 additional patients and collect 6-month follow-up data.
"""
}

# Create the sample files
for file_path, content in alice_files.items():
    create_sample_file(file_path, content)

Created sample file: /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/patient_data.csv
Created sample file: /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/research_notes.txt


## Setting File Permissions for Bob

In [4]:
# Set permissions for Bob to access Alice's patient data
bob_permissions = alice_client.set_file_permissions(
    to_email=alice_client.client.email,  # Alice setting permissions on her own server
    user_email="bob@openmined.org",  # For Bob
    file_paths=[f"{alice_data_dir}/patient_data.csv"],  # Only allow access to this file
    operation="set"
)

print(f"Set Bob's permissions on Alice's server: {bob_permissions.success}")
print(f"Bob can access: {bob_permissions.allowed_files}")

Set Bob's permissions on Alice's server: True
Bob can access: ['/Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/patient_data.csv']


## Testing Alice's Client with Own Data

In [5]:
prompt = "Analyze the patient data in the file. What conditions are being treated? What's the age distribution?"

response = alice_client.generate(
    to_email=alice_client.client.email,  # Alice querying her own data
    model='llama3',
    prompt=prompt,
    system="You are a healthcare research assistant. Provide analytical insights based on the data.",
    temperature=0.7,
    files=[f"{alice_data_dir}/patient_data.csv"]  # Using Alice's file
)

print(f"\nResponse from Alice's LLM on her own data:\n{'-'*40}\n{response.response}\n{'-'*40}")


Response from Alice's LLM on her own data:
----------------------------------------
Error: User alice@openmined.org does not have permission to access file: /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/patient_data.csv
----------------------------------------
